In [4]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
filename = 'results-survey976727_pre.csv'
data = pd.read_csv(filename, sep=';', header=0)

data.drop(['submitdate', 'G00Q01[SQ01]', 'G02Q01', 'G02Q02', 'G02Q03', 'G02Q04', 'G03Q02[other]', 'id'], axis = 'columns', inplace=True)

data

In [ ]:
# Calcular los límites de los valores atípicos
# regla empírica, que establece que los valores atípicos se encuentran a más de 3 desviaciones estándar de la media.

edad_media = data['G03Q01'].mean()
edad_desv_est = data['G03Q01'].std()
edad_limite_inferior = edad_media - (3*edad_desv_est)
edad_limite_superior = edad_media + (3*edad_desv_est)

# Filtrar el dataframe para eliminar los outliers de edad
data_filtrado = data.loc[(data['G03Q01'] >= edad_limite_inferior) & (data['G03Q01'] <= edad_limite_superior)]

data_filtrado

In [ ]:
# Crear un DataFrame con los datos de los intentos medios de cada jugador en cada nivel
intentosJugadores = pd.DataFrame(intentosMedios_Individual["intentosIndividual"])

# Pivotear el DataFrame para tener un formato largo
intentos_df_long = pd.melt(intentosJugadores.reset_index(), id_vars=['index'], var_name='Jugador', value_name='Intentos')
intentos_df_long.columns = ['Nivel', 'Jugador', 'Intentos']

# Agrupar los intentos de cada jugador por nivel y tomar la media
intentos_jugador_nivel = intentos_df_long.groupby(['Jugador', 'Nivel'])['Intentos'].mean().unstack()

# Unir la tabla de intentos de cada jugador con los datos originales
df = data_filtrado.merge(intentos_jugador_nivel, left_on='token', right_index=True)

# Seleccionar las variables a utilizar para el clustering
X = df.iloc[:, 3:] # Seleccionar las columnas correspondientes a los niveles

# Imputar los valores faltantes
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Normalizar las variables utilizando StandardScaler
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_imputed)

# Definir el número de clusters y entrenar el modelo de K-means
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(X_norm)

# Asignar a cada jugador el cluster correspondiente
clusters = kmeans.labels_
df['Cluster'] = clusters

# Calcular la media de los números de intentos en cada cluster
means = df.groupby(['G03Q02', 'G03Q01', 'Cluster'])[X.columns].mean()

# Mostrar los resultados
print(means)
#intentosJugadores


In [ ]:
tiemposJugadores = pd.DataFrame(tiemposMedios["tiemposIndividuales"])

# Aplicar la función int() solamente a los valores que no son NaN en el DataFrame
tiemposJugadores = tiemposJugadores.applymap(lambda x: int(x) if x == x else x)

# Pivotear el DataFrame para tener un formato largo
tiempos_df_long = pd.melt(tiemposJugadores.reset_index(), id_vars=['index'], var_name='Jugador', value_name='Intentos')
tiempos_df_long.columns = ['Nivel', 'Jugador', 'Intentos']

# Agrupar los intentos de cada jugador por nivel y tomar la media
tiempos_jugador_nivel = tiempos_df_long.groupby(['Jugador', 'Nivel'])['Intentos'].mean().unstack()

# Unir la tabla de intentos de cada jugador con los datos originales
df = data_filtrado.merge(tiempos_jugador_nivel, left_on='token', right_index=True)

# Seleccionar las variables a utilizar para el clustering
X = df.iloc[:, 3:] # Seleccionar las columnas correspondientes a los niveles

# Imputar los valores faltantes
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Normalizar las variables utilizando StandardScaler
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_imputed)

# Definir el número de clusters y entrenar el modelo de K-means
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(X_norm)

# Asignar a cada jugador el cluster correspondiente
clusters = kmeans.labels_
df['Cluster'] = clusters

# Calcular la media de los números de intentos en cada cluster
means = df.groupby(['G03Q02', 'G03Q01', 'Cluster'])[X.columns].mean()

# Mostrar los resultados
print(means)
# tiemposJugadores


In [ ]:
media_intentos = df.groupby(['G03Q02', 'G03Q01', 'Cluster'])[X.columns].mean().reset_index()
media_intentos = pd.melt(media_intentos, id_vars=['G03Q02', 'G03Q01', 'Cluster'], var_name='Nivel', value_name='Intentos')

import seaborn as sns
import matplotlib.pyplot as plt

# Crear una figura para cada nivel
for nivel in intentos_df_long['Nivel'].unique():
    # Seleccionar los datos del nivel y generar una tabla pivote con los valores medios de intentos por género
    df_nivel = media_intentos[media_intentos['Nivel'] == nivel]
    pivot_table = pd.pivot_table(df_nivel, values='Intentos', index=['G03Q01'], columns=['Cluster'], aggfunc=np.mean)
    
    # Crear un gráfico de barras apiladas para comparar los valores medios de intentos por género en cada cluster
    sns.catplot(x='G03Q01', y='Intentos', hue='Cluster', data=df_nivel, kind='bar', ci=None, legend_out=False)
    plt.title(f'Nivel {nivel}')
    plt.show()
